In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt
import torch
from torch import optim, nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision.utils import make_grid
from torchvision import transforms as T
from torchvision import models, datasets
import os
from random import randint
import urllib
import zipfile
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torchsummary import summary
import pickle
from torchvision.utils import save_image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Create Model

we create a class that inverts the alexnet. we get the layer we want to recreate images as l. we seperate the layers we want and freez the weights. we create 3 models for the 3 layers. after passing the data through the alexnet, we pass its output through the model we defined.

if the layer is convolutional, we set the whole model to the selected features. if the layer is fc, we only change the fc layers so that the conv layers would remain unchanged



In [4]:
class InverseNet(nn.Module):

    def __init__(self, l):
        super(InverseNet, self).__init__()
        self.an = models.alexnet(pretrained=True)
        self.cut_layer=l
        feats = self.an.features
        cl = self.an.classifier
        if l <= 5:
            split_point = 0
            conv_cntr = 0
            for lay in feats:
                if isinstance(lay, nn.Conv2d):
                    conv_cntr += 1
                split_point += 1

                if conv_cntr == l:
                    break
                
            self.an= nn.Sequential(*list(feats)[:split_point+2])
            
                
        elif l >=6 and l <=8:
            split_point = 0
            lin_cntr = 5
            for lay in feats:
                if isinstance(lay, nn.Linear):
                    lin_cntr += 1
                split_point += 1

                if lin_cntr == l:
                    break
                
            self.an.cl = nn.Sequential(*list(cl)[:split_point+1])

        # Freeze base network parameters
        for param in self.an.parameters():
            param.requires_grad = False

        self.lin_fc8 = nn.Sequential(nn.Linear(1000, 4096),
                                     nn.LeakyReLU(0.2, inplace=True),
                                     nn.Linear(4096, 4096),
                                     nn.LeakyReLU(0.2, inplace=True),
                                     nn.Linear(4096, 2048),
                                     nn.LeakyReLU(0.2, inplace=True))
        
        self.conv_fc8 = nn.Sequential(nn.ConvTranspose2d(128, 128, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(128, 64, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(64, 32, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(32, 16, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(16, 3, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True))
        
        self.conv_conv5 = nn.Sequential(nn.ConvTranspose2d(256, 128, 3, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(128, 64, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(64, 32, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(32, 16, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(16, 8, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(8, 3, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True))
        
        self.conv_conv2 = nn.Sequential(nn.ConvTranspose2d(192, 128, 7, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(128, 64, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(64, 32, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True),
                                      nn.ConvTranspose2d(32, 3, 5, stride=2, padding=2, output_padding=1),
                                      nn.LeakyReLU(0.2, inplace=True))
    
    def forward(self, x):
        x = self.an(x)
        if self.cut_layer == 5:
            x = self.conv_conv5(x)
        elif self.cut_layer == 2:
            x = self.conv_conv2(x)
        elif self.cut_layer == 8:
            x = self.lin_fc8(x)
            x = x.view(x.size(0), 128, 4, 4)
            x = self.conv_fc8(x)
        return x


# Create Dataset

we define a function to transform and label the dataset based on the folder names, and loads batches of data.

In [ ]:
def generate_dataloader(data, name,batch_size,input_size):

    transform = transforms.Compose([transforms.Resize(input_size),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
 
    dataset = datasets.ImageFolder(data, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size,shuffle=(name=="train"))
    
    return dataloader

for the selceted images for part C, we extract and inverse the features using the trained models and save the output image

In [ ]:
def test_image(model,folder_path,all_data,name):
  for i, data in enumerate(all_data):
    img, cl = data
    img, cl = img.to(device), cl.to(device)
    out = model(img)

    image_name='original'+str(i)+'.png'
    image_path=os.path.join(folder_path,image_name)
    out_name=name+'_recreat'+str(i)+'.png'
    out_path=os.path.join(folder_path,out_name)

    save_image(img, image_path)
    save_image(out, out_path)

# Train model

**Loading data**

In [ ]:
train_path='/content/drive/MyDrive/Untitled folder/selected_class_train'
test_path='/content/drive/MyDrive/selected_class_test'
batch_size = 128
input_size=(128,128)
train_data = generate_dataloader(train_path, "train",batch_size,input_size)
test_data = generate_dataloader(test_path, "test", batch_size,input_size)

In [ ]:
show_path='/content/drive/MyDrive/Untitled folder/selected_images'
data_show=generate_dataloader(show_path, "show",batch_size,input_size)

**Conv2**

In [14]:
model_conv2 = InverseNet(2)
model_conv2.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda x: x.requires_grad, model_conv2.parameters()))

In [16]:
print("conv2")
summary(model_conv2,(3,128,128))

conv2
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 31, 31]          23,296
              ReLU-2           [-1, 64, 31, 31]               0
         MaxPool2d-3           [-1, 64, 15, 15]               0
            Conv2d-4          [-1, 192, 15, 15]         307,392
              ReLU-5          [-1, 192, 15, 15]               0
         MaxPool2d-6            [-1, 192, 7, 7]               0
   ConvTranspose2d-7          [-1, 128, 16, 16]       1,204,352
         LeakyReLU-8          [-1, 128, 16, 16]               0
   ConvTranspose2d-9           [-1, 64, 32, 32]         204,864
        LeakyReLU-10           [-1, 64, 32, 32]               0
  ConvTranspose2d-11           [-1, 32, 64, 64]          51,232
        LeakyReLU-12           [-1, 32, 64, 64]               0
  ConvTranspose2d-13          [-1, 3, 128, 128]           2,403
        LeakyReLU-14          [-1

In [ ]:
epochs=10
all_train_loss=[]
all_test_loss=[]
for e in range(epochs):
        loss_train = 0.0
        loss_test = 0.0
        for i, data in enumerate(train_data):
            img, cl = data
            img, cl = img.to(device), cl.to(device)
            optimizer.zero_grad()
            out = model_conv2(img)         
            loss = criterion(out, img)
            loss.backward()
            loss_train += loss.item()
            optimizer.step()
            print(i)
            print("{0:.4}".format(loss))
            
        all_train_loss.append(loss_train/len(train_data))
        for i, data in enumerate(test_data):
            img, cl = data
            img, cl = img.to(device), cl.to(device)
            out = model_conv2(img)         
            loss = criterion(out, img)
            print(i)
            print("{0:.4}".format(loss))
            loss_test += loss.item()
        all_test_loss.append(loss_test/len(test_data))
        print("##################################")
        print("epoch: "+str(e))
        print("train_loss {0:.4}".format(loss_train/len(train_data)))
        print("test_loss {0:.4}".format(loss_test/len(test_data)))
        print("##################################")
        



            

In [ ]:

model_conv2_path='/content/drive/MyDrive/model_conv2/conv2.pth'
torch.save({
            'epoch': epochs,
            'model_state_dict': model_conv2.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, model_conv2_path)


with open('/content/drive/MyDrive/model_conv2/conv2_train_loss.pkl', 'wb') as f:
    pickle.dump(all_train_loss, f)

with open('/content/drive/MyDrive/model_conv2/conv2_test_loss.pkl', 'wb') as f:
    pickle.dump(all_test_loss, f)

**Conv5**

In [17]:
model_conv5 = InverseNet(5)
model_conv5.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda x: x.requires_grad, model_conv5.parameters()))

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
print("conv5")
summary(model_conv5,(3,128,128))

conv5
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 31, 31]          23,296
              ReLU-2           [-1, 64, 31, 31]               0
         MaxPool2d-3           [-1, 64, 15, 15]               0
            Conv2d-4          [-1, 192, 15, 15]         307,392
              ReLU-5          [-1, 192, 15, 15]               0
         MaxPool2d-6            [-1, 192, 7, 7]               0
            Conv2d-7            [-1, 384, 7, 7]         663,936
              ReLU-8            [-1, 384, 7, 7]               0
            Conv2d-9            [-1, 256, 7, 7]         884,992
             ReLU-10            [-1, 256, 7, 7]               0
           Conv2d-11            [-1, 256, 7, 7]         590,080
             ReLU-12            [-1, 256, 7, 7]               0
        MaxPool2d-13            [-1, 256, 3, 3]               0
  ConvTranspose2d-14            [

In [ ]:
epochs=15
all_train_loss=[]
all_test_loss=[]
for e in range(epochs):
        loss_train = 0.0
        loss_test = 0.0
        for i, data in enumerate(train_data):
            img, cl = data
            img, cl = img.to(device), cl.to(device)
            optimizer.zero_grad()
            out = model_conv5(img)         
            loss = criterion(out, img)
            loss.backward()
            loss_train += loss.item()
            optimizer.step()
            print(i)
            print("{0:.4}".format(loss))
            
        all_train_loss.append(loss_train/len(train_data))
        for i, data in enumerate(test_data):
            img, cl = data
            img, cl = img.to(device), cl.to(device)
            out = model_conv5(img)         
            loss = criterion(out, img)
            print(i)
            print("{0:.4}".format(loss))
            loss_test += loss.item()
        all_test_loss.append(loss_test/len(test_data))
        print("##################################")
        print("epoch: "+str(e))
        print("train_loss {0:.4}".format(loss_train/len(train_data)))
        print("test_loss {0:.4}".format(loss_test/len(test_data)))
        print("##################################")
        



            

In [ ]:
model_conv5_path='/content/drive/MyDrive/model_conv5/conv5.pth'
torch.save({
            'epoch': epochs,
            'model_state_dict': model_conv5.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, model_conv5_path)


with open('/content/drive/MyDrive/model_conv5/conv5_train_loss.pkl', 'wb') as f:
    pickle.dump(all_train_loss, f)

with open('/content/drive/MyDrive/model_conv5/conv5_test_loss.pkl', 'wb') as f:
    pickle.dump(all_test_loss, f)

**FC8**

In [19]:
model_fc8 = InverseNet(8)
model_fc8.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda x: x.requires_grad, model_fc8.parameters()))

In [20]:
print("fc8")
summary(model_fc8,(3,128,128))

fc8
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 31, 31]          23,296
              ReLU-2           [-1, 64, 31, 31]               0
         MaxPool2d-3           [-1, 64, 15, 15]               0
            Conv2d-4          [-1, 192, 15, 15]         307,392
              ReLU-5          [-1, 192, 15, 15]               0
         MaxPool2d-6            [-1, 192, 7, 7]               0
            Conv2d-7            [-1, 384, 7, 7]         663,936
              ReLU-8            [-1, 384, 7, 7]               0
            Conv2d-9            [-1, 256, 7, 7]         884,992
             ReLU-10            [-1, 256, 7, 7]               0
           Conv2d-11            [-1, 256, 7, 7]         590,080
             ReLU-12            [-1, 256, 7, 7]               0
        MaxPool2d-13            [-1, 256, 3, 3]               0
AdaptiveAvgPool2d-14            [-1

In [ ]:
epochs=20
all_train_loss=[]
all_test_loss=[]
for e in range(epochs):
        loss_train = 0.0
        loss_test = 0.0
        for i, data in enumerate(train_data):
            img, cl = data
            img, cl = img.to(device), cl.to(device)
            optimizer.zero_grad()
            out = model_fc8(img)         
            loss = criterion(out, img)
            loss.backward()
            loss_train += loss.item()
            optimizer.step()
            print(i)
            print("{0:.4}".format(loss))
            
        all_train_loss.append(loss_train/len(train_data))
        for i, data in enumerate(test_data):
            img, cl = data
            img, cl = img.to(device), cl.to(device)
            out = model_fc8(img)         
            loss = criterion(out, img)
            print(i)
            print("{0:.4}".format(loss))
            loss_test += loss.item()
        all_test_loss.append(loss_test/len(test_data))
        print("##################################")
        print("epoch: "+str(e))
        print("train_loss {0:.4}".format(loss_train/len(train_data)))
        print("test_loss {0:.4}".format(loss_test/len(test_data)))
        print("##################################")
        



            

In [ ]:
model_fc8_path='/content/drive/MyDrive/model_fc8/fc8.pth'
torch.save({
            'epoch': epochs,
            'model_state_dict': model_fc8.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, model_fc8_path)


with open('/content/drive/MyDrive/model_fc8/fc8_train_loss.pkl', 'wb') as f:
    pickle.dump(all_train_loss, f)

with open('/content/drive/MyDrive/model_fc8/fc8_test_loss.pkl', 'wb') as f:
    pickle.dump(all_test_loss, f)

#B

In [3]:
with open('/content/drive/MyDrive/model_conv2/conv2_train_loss.pkl', 'rb') as f:
      conv2_train_loss=pickle.load( f)

with open('/content/drive/MyDrive/model_conv2/conv2_test_loss.pkl', 'rb') as f:
      conv2_test_loss=pickle.load( f)   

with open('/content/drive/MyDrive/model_conv5/conv5_train_loss.pkl', 'rb') as f:
      conv5_train_loss=pickle.load( f)

with open('/content/drive/MyDrive/model_conv5/conv5_test_loss.pkl', 'rb') as f:
      conv5_test_loss=pickle.load( f) 

with open('/content/drive/MyDrive/model_fc8/fc8_train_loss.pkl', 'rb') as f:
      fc8_train_loss=pickle.load( f)

with open('/content/drive/MyDrive/model_fc8/fc8_test_loss.pkl', 'rb') as f:
      fc8_test_loss=pickle.load( f) 

In [4]:
def show_results( all_loss_train, all_loss_test, model_name):
  

  n=len(all_loss_test)
  plt.figure(figsize=(n*8, n*6)) 
  for i in range(n):
      plt.subplot(3, 2, 2*i+1)
      plt.plot(all_loss_train[i])
      plt.xlabel('Epoch')
      plt.ylabel('L2 Loss')
      plt.title('Training Loss vs Epoch in '+model_name[i])
      

      plt.subplot(3, 2, 2*i+2)
      plt.plot(all_loss_test[i])
      plt.xlabel('Epoch')
      plt.ylabel('L2 Loss')
      plt.title('Evaluation Loss vs Epoch in '+model_name[i])
  
  plt.suptitle('Comparing Training and Evaluation Loss vs Epoch in Different Models ', fontsize = 15)
      



In [5]:
all_loss_train=[conv2_train_loss, conv5_train_loss, fc8_train_loss]
all_loss_test=[conv2_test_loss, conv5_test_loss, fc8_test_loss]
model_name=['Conv2', 'Conv5', 'Fc8']

In [ ]:
show_results( all_loss_train, all_loss_test, model_name)

In [6]:
train_conv2_ss=sum(conv2_train_loss[7:10])/3
valid_conv2_ss=sum(conv2_test_loss[7:10])/3

train_conv5_ss=sum(conv5_train_loss[12:15])/3
valid_conv5_ss=sum(conv5_test_loss[12:15])/3

train_fc8_ss=sum(fc8_train_loss[17:20])/3
valid_fc8_ss=sum(fc8_test_loss[17:20])/3

df=pd.DataFrame({"layer":["conv2d","conv5d","fc8"],"train error":[train_conv2_ss, train_conv5_ss, train_fc8_ss],
                 "valid error":[valid_conv2_ss, valid_conv5_ss, valid_fc8_ss]})
df

,layer,train error,valid error
0,conv2d,0.049827,0.053309
1,conv5d,0.116861,0.141264
2,fc8,0.175442,0.195052


# C

we create a dataset from the chosen images. we set the batch size to 1

In [ ]:
show_path='/content/drive/MyDrive/chosen_images'
data_show=generate_dataloader(show_path, "show",1,input_size)

In [ ]:
name="conv2"
con2_path='/content/drive/MyDrive/conv2_image'
test_image(model_conv2,con2_path,data_show,name)

In [ ]:
name="conv5"
con5_path='/content/drive/MyDrive/conv5_image'
test_image(model_conv5,con5_path,data_show,name)

In [ ]:
name="fc8"
fc8_path='/content/drive/MyDrive/fc8_image'
test_image(model_fc8,fc8_path,data_show,name)